In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [3]:
import os
os.environ["OPENAI_API_KEY"] = ""
#os.environ["SERPAPI_API_KEY"] = ""

In [4]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('/Users/sushilaverma/Downloads/Appointment Receipt.pdf')

In [5]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [6]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [7]:
len(texts)

27

In [8]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [22]:
document_search = FAISS.from_texts(texts, embeddings)

In [23]:
document_search

In [24]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [25]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [29]:
query = "What is the Appointment date"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The appointment date is 07/12/2023.'

In [32]:
query = "What is the Mother's Name of the applicant"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\nSANT OSH KUMARI'

In [33]:
# Edge Case, we should be specifically mentioning what we would like to search, otherwise it will prompt the generic result.
query = "What is the Mother's Name"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

"\nThe mother's name should be listed on the birth certificate or a declaration signed by the parents. If the mother's name is not available, it may be listed on other documents such as the transfer/school leaving/matriculation certificate or policy bond. If none of these documents list the mother's name, it may be necessary to provide an affidavit stating the reason for the absence of the mother's name."